In [10]:
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import json
import os 
import sys 

In [11]:
filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable/TransientTable_gw170817.csv"
#filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable_gw170817.json"

dataframe = pd.read_csv(filename, sep=";")
dataframe.head()

,Unnamed: 0,ampel_z,ampel_z_precision,ampel_dist,possis_abspeak,possis_chisq,possis_ndof,chisquares,grb_spatial,kilonovaness,...,map_dist,map_dist_unc,map_name,trigger_time,map_seed,stock,ztf_id,ra,dec,channels
0,14,0.00968,0.003,9.947793,-15.142820,486.264726,13,"{'bns_maurice_1': {'chisq': 1310.877799534361,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
1,12,0.00968,0.003,9.947793,-15.139010,393.356226,11,{'bns_maurice_1': {'chisq': 1054.1621576432042...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
2,13,0.00968,0.003,9.947793,-15.140046,418.500264,12,{'bns_maurice_1': {'chisq': 1122.6030874685073...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
3,11,0.00968,0.003,9.947793,-15.138362,388.871599,10,{'bns_maurice_1': {'chisq': 1019.7583765326634...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
4,9,0.00968,0.003,9.947793,-15.132537,282.114018,8,"{'bns_maurice_1': {'chisq': 725.3279466693862,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817


In [12]:
def jsonify(string: str):
    tmp_str = string.replace("'", '"')
    tmp_str = tmp_str.replace("None", "null")
    tmp_str = tmp_str.replace("True", "true")
    tmp_str = tmp_str.replace("False", "false")

    return tmp_str

In [13]:
risedec_df = pd.DataFrame()

pos_rowslist_dict = {"bns_maurice_1": [], "bns_maurice_2": [], "bns_maurice_3": [], "bns_maurice_4": []}

rowslist = []
for k, row in dataframe.iterrows():
    dec_str = jsonify(row["rise_decline"])
    #print(tmp_str)
    dec_dict = json.loads(dec_str)
    #print(dec_dict)

    rowslist.append(dec_dict)

    pos_str = jsonify(row["chisquares"])
    pos_dict = json.loads(pos_str)
    #print(pos_dict)
    for key, value in pos_dict.items():
        value["ndet"] = dec_dict["ndet"]
        value["latest_det"] = dec_dict["jd_last"]
        value["first_det"] = dec_dict["jd_det"]
        value["kilonovaness"] = row["kilonovaness"]
        pos_rowslist_dict[key].append(value)
    #print(row["rise_decline"]["ndet"])

risedec_df = pd.DataFrame(rowslist)

pos_df_dict = {}
for key in pos_rowslist_dict:
    pos_df_dict[key] = pd.DataFrame(pos_rowslist_dict[key]).sort_values(by="ndet")


In [14]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - pos_df_dict[key]["first_det"], y = pos_df_dict[key]["kilonovaness"], name=key))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "kilonovaness"})
fig.show()

In [15]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - pos_df_dict[key]["first_det"], y = pos_df_dict[key]["chisq"]/pos_df_dict[key]["ndof"], name=key))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "Reduced chisqu"})
fig.show()

In [16]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - pos_df_dict[key]["first_det"], y = pos_df_dict[key]["obspeak_model_B"], name=key))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "Reduced chisqu"})
fig.show()